# **Feature engineering my crypto data**

In this *notebook*, I will add some important *features* to my data, that initially has only *prices* (close, high, low and open) and *volume*, which doesn't offer that much info to train a model to predict future prices.

In [1]:
import pandas as pd
import numpy as np

Let's load and have a peak into our data

In [2]:
btc = pd.read_csv('coins_data/btc.csv')
eth = pd.read_csv('coins_data/eth.csv')
sol = pd.read_csv('coins_data/sol.csv')
bnb = pd.read_csv('coins_data/bnb.csv')
eth.head(10)

,Price,Close,High,Low,Open,Volume
0,Ticker,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
1,Date,NaN,NaN,NaN,NaN,NaN
2,2017-11-09,320.8840026855469,329.4519958496094,307.0559997558594,308.6449890136719,893249984
3,2017-11-10,299.25299072265625,324.7179870605469,294.5419921875,320.6709899902344,885985984
4,2017-11-11,314.6809997558594,319.4530029296875,298.1919860839844,298.58599853515625,842300992
5,2017-11-12,307.9079895019531,319.15301513671875,298.51300048828125,314.69000244140625,1613479936
6,2017-11-13,316.71600341796875,328.4150085449219,307.0249938964844,307.0249938964844,1041889984
7,2017-11-14,337.6310119628906,340.177001953125,316.76300048828125,316.76300048828125,1069680000
8,2017-11-15,333.35699462890625,340.9119873046875,329.81298828125,337.9639892578125,722665984
9,2017-11-16,330.92401123046875,336.15899658203125,323.6059875488281,333.4429931640625,797254016


It seems that our data needs some cleaning

In [3]:
columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

btc = btc[2:]
eth = eth[2:]
sol = sol[2:]
bnb = bnb[2:]

btc.columns = columns
eth.columns = columns
sol.columns = columns
bnb.columns = columns

btc.reset_index(inplace=True)
eth.reset_index(inplace=True)
sol.reset_index(inplace=True)
bnb.reset_index(inplace=True)

btc.head()

,index,Date,Close,High,Low,Open,Volume
0,2,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
1,3,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
2,4,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
3,5,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600
4,6,2014-09-21,398.8210144042969,412.4259948730469,393.1809997558594,408.0849914550781,26580100


In [4]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2691 entries, 0 to 2690
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   2691 non-null   int64 
 1   Date    2691 non-null   object
 2   Close   2691 non-null   object
 3   High    2691 non-null   object
 4   Low     2691 non-null   object
 5   Open    2691 non-null   object
 6   Volume  2691 non-null   object
dtypes: int64(1), object(6)
memory usage: 147.3+ KB


It seems all good now!

## **1. Moving averages (MA)**

The first feature we'll 'engineer', is moving averages.

Moving averages in crypto trading smooth out price data to identify trends by calculating the average price over a specific period. They help traders spot potential buy/sell signals, support/resistance levels, and trend reversals.

### **a. 3-day MA**

We'll start with a 3 day moving average, which is the average of the previous 3 days prices, we'll go with the closing price

In [5]:
#Starting with bitcoin

btc['Close'] = pd.to_numeric(btc['Close'], errors='coerce')
btc['3d-MA'] = np.nan

btc.loc[0, '3d-MA'] = 0
btc.loc[1, '3d-MA'] = btc.loc[0, 'Close']
btc.loc[2, '3d-MA'] = 0.5 * (btc.loc[0, 'Close'] + btc.loc[1, 'Close'])

for i in range(3, len(btc)):
    btc.loc[i, '3d-MA'] = (1/3) * (btc.loc[i-1, 'Close'] + btc.loc[i-2, 'Close'] + btc.loc[i-3, 'Close'])
    
btc.head()

,index,Date,Close,High,Low,Open,Volume,3d-MA
0,2,2014-09-17,457.334015,468.17401123046875,452.4219970703125,465.864013671875,21056800,0.000000
1,3,2014-09-18,424.440002,456.8599853515625,413.10400390625,456.8599853515625,34483200,457.334015
2,4,2014-09-19,394.795990,427.8349914550781,384.5320129394531,424.1029968261719,37919700,440.887009
3,5,2014-09-20,408.903992,423.2959899902344,389.88299560546875,394.6730041503906,36863600,425.523336
4,6,2014-09-21,398.821014,412.4259948730469,393.1809997558594,408.0849914550781,26580100,409.379995


And now to do the same for ETH, SOL and BNB:

In [6]:
eth['Close'] = pd.to_numeric(eth['Close'], errors='coerce')
eth['3d-MA'] = np.nan

eth.loc[0, '3d-MA'] = 0
eth.loc[1, '3d-MA'] = eth.loc[0, 'Close']
eth.loc[2, '3d-MA'] = 0.5 * (eth.loc[0, 'Close'] + eth.loc[1, 'Close'])

for i in range(3, len(eth)):
    eth.loc[i, '3d-MA'] = (1/3) * (eth.loc[i-1, 'Close'] + eth.loc[i-2, 'Close'] + eth.loc[i-3, 'Close'])

sol['Close'] = pd.to_numeric(sol['Close'], errors='coerce')
sol['3d-MA'] = np.nan

sol.loc[0, '3d-MA'] = 0
sol.loc[1, '3d-MA'] = sol.loc[0, 'Close']
sol.loc[2, '3d-MA'] = 0.5 * (sol.loc[0, 'Close'] + sol.loc[1, 'Close'])

for i in range(3, len(sol)):
    sol.loc[i, '3d-MA'] = (1/3) * (sol.loc[i-1, 'Close'] + sol.loc[i-2, 'Close'] + sol.loc[i-3, 'Close'])

bnb['Close'] = pd.to_numeric(bnb['Close'], errors='coerce')
bnb['3d-MA'] = np.nan

bnb.loc[0, '3d-MA'] = 0
bnb.loc[1, '3d-MA'] = bnb.loc[0, 'Close']
bnb.loc[2, '3d-MA'] = 0.5 * (bnb.loc[0, 'Close'] + bnb.loc[1, 'Close'])

for i in range(3, len(bnb)):
    bnb.loc[i, '3d-MA'] = (1/3) * (bnb.loc[i-1, 'Close'] + bnb.loc[i-2, 'Close'] + bnb.loc[i-3, 'Close'])
    
sol.head()

,index,Date,Close,High,Low,Open,Volume,3d-MA
0,2,2020-04-10,0.951054,1.3134870529174805,0.6941869854927063,0.8320050239562988,87364276,0.000000
1,3,2020-04-11,0.776819,1.0490729808807373,0.7650200128555298,0.9510539770126343,43862444,0.951054
2,4,2020-04-12,0.882507,0.9566699862480164,0.7624260187149048,0.7854480147361755,38736897,0.863936
3,5,2020-04-13,0.777832,0.8916029930114746,0.7739760279655457,0.8907600045204163,18211285,0.870127
4,6,2020-04-14,0.661925,0.7964720129966736,0.6281690001487732,0.7778319716453552,16747614,0.812386


All good, now to quickly do the other moving averages periods (10, 50 and 100 days)

In [7]:
# 10 days moving average

coins = [btc, eth, sol, bnb]

for coin in coins:
    coin['10d-MA'] = np.nan
    for day in range(1,len(coin)):
        somme = 0
        prev = min(10,day)
        for dayup in range(day-prev,day):
            somme += coin.loc[dayup, 'Close']
        coin.loc[day, '10d-MA'] = somme / prev

eth.tail()

,index,Date,Close,High,Low,Open,Volume,3d-MA,10d-MA
2686,2688,2025-03-18,1932.543457,1935.1820068359375,1872.5089111328125,1927.0057373046875,10170844746,1917.277547,1943.126819
2687,2689,2025-03-19,2057.749023,2068.764404296875,1928.24951171875,1932.5445556640625,20065206266,1915.660482,1916.230139
2688,2690,2025-03-20,1982.099854,2067.483642578125,1952.2432861328125,2057.951171875,13217865782,1972.431803,1920.454126
2689,2691,2025-03-21,1964.847534,1994.8914794921875,1937.2115478515625,1981.85302734375,9708125480,1990.797445,1932.548975
2690,2692,2025-03-22,1980.037842,2005.0411376953125,1964.27099609375,1964.9404296875,6117036512,2001.565470,1937.049231


And now to plot and visualise these new features, using candlesticks for the prices and line for the MA

In [8]:
# Importing necessary libs

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
from mpl_finance import candlestick_ohlc

/home/okamitah/Projects/trading-bot/.venv/lib/python3.13/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [16]:
# Plotting for btc for the last 30 days

btc_data = btc[['Date', 'Open', 'High', 'Low', 'Close']].tail(30).copy()
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data = btc_data.dropna(subset=['Date'])

btc_3d = btc['3d-MA'].tail(30).copy()
btc_10d = btc['10d-MA'].tail(30).copy()

btc_data['Open'] = pd.to_numeric(btc_data['Open'], errors='coerce')
btc_data['High'] = pd.to_numeric(btc_data['High'], errors='coerce')
btc_data['Low'] = pd.to_numeric(btc_data['Low'], errors='coerce')
btc_data['Close'] = pd.to_numeric(btc_data['Close'], errors='coerce')

btc_data['Date'] = btc_data['Date'].map(mdates.date2num)

ax = plt.subplot()
ax.xaxis_date()

btc_values = btc_data[['Date', 'Open', 'High', 'Low', 'Close']].values
candlestick_ohlc(ax, btc_values, width=0.6, colorup='g', colordown='r')

plt.plot(btc_data['Date'],btc_3d)
plt.plot(btc_data['Date'],btc_10d)

plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [17]:
# Plotting for eth for the last 20 days

eth_data = eth[['Date', 'Open', 'High', 'Low', 'Close']].tail(20).copy()

eth_data['Date'] = pd.to_datetime(eth_data['Date'])

eth_data = eth_data.dropna(subset=['Date'])

eth_3d = eth['3d-MA'].tail(20).copy()
eth_10d = eth['10d-MA'].tail(20).copy()

eth_data['Open'] = pd.to_numeric(eth_data['Open'], errors='coerce')
eth_data['High'] = pd.to_numeric(eth_data['High'], errors='coerce')
eth_data['Low'] = pd.to_numeric(eth_data['Low'], errors='coerce')
eth_data['Close'] = pd.to_numeric(eth_data['Close'], errors='coerce')


eth_data['Date'] = eth_data['Date'].map(mdates.date2num)

ax = plt.subplot()
ax.xaxis_date()

eth_values = eth_data[['Date', 'Open', 'High', 'Low', 'Close']].values

candlestick_ohlc(ax, eth_values, width=0.6, colorup='g', colordown='r')

plt.plot(eth_data['Date'],eth_3d)
plt.plot(eth_data['Date'],eth_10d)

plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [19]:
# Plotting sol for the last 25 days

sol_data = sol[['Date', 'Open', 'High', 'Low', 'Close']].tail(25).copy()

sol_data['Date'] = pd.to_datetime(sol_data['Date'])

sol_data = sol_data.dropna(subset=['Date'])

sol_3d = sol['3d-MA'].tail(25).copy()
sol_10d = sol['10d-MA'].tail(25).copy()

sol_data['Open'] = pd.to_numeric(sol_data['Open'], errors='coerce')
sol_data['High'] = pd.to_numeric(sol_data['High'], errors='coerce')
sol_data['Low'] = pd.to_numeric(sol_data['Low'], errors='coerce')
sol_data['Close'] = pd.to_numeric(sol_data['Close'], errors='coerce')


sol_data['Date'] = sol_data['Date'].map(mdates.date2num)

ax = plt.subplot()
ax.xaxis_date()

sol_values = sol_data[['Date', 'Open', 'High', 'Low', 'Close']].values

candlestick_ohlc(ax, sol_values, width=0.6, colorup='g', colordown='r')

plt.plot(sol_data['Date'],sol_3d)
plt.plot(sol_data['Date'],sol_10d)

plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [23]:
# Plotting bnb for the last 35 days

bnb_data = bnb[['Date', 'Open', 'High', 'Low', 'Close']].tail(35).copy()

bnb_data['Date'] = pd.to_datetime(bnb_data['Date'])

bnb_data = bnb_data.dropna(subset=['Date'])

bnb_3d = bnb['3d-MA'].tail(35).copy()
bnb_10d = bnb['10d-MA'].tail(35).copy()

bnb_data['Open'] = pd.to_numeric(bnb_data['Open'], errors='coerce')
bnb_data['High'] = pd.to_numeric(bnb_data['High'], errors='coerce')
bnb_data['Low'] = pd.to_numeric(bnb_data['Low'], errors='coerce')
bnb_data['Close'] = pd.to_numeric(bnb_data['Close'], errors='coerce')

bnb_data['Date'] = bnb_data['Date'].map(mdates.date2num)

ax = plt.subplot()
ax.xaxis_date()

bnb_values = bnb_data[['Date', 'Open', 'High', 'Low', 'Close']].values

candlestick_ohlc(ax, bnb_values, width=0.6, colorup='g', colordown='r')

plt.plot(bnb_data['Date'],bnb_3d, color='blue', label='3D-MA')
plt.plot(bnb_data['Date'],bnb_10d, color='orange', label='10D-MA')

plt.legend()

plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

## **2.**